<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EA%B8%B0%EC%B4%88(%EC%8B%9C%ED%80%B8%EC%8A%A4_%EB%AA%A8%EB%8D%B8%EB%A7%81%2C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98)_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사전학습 임베딩 적용(Word2Vec, FastText)
```
사전학습 임베딩
  대량의 데이터로 학습된 단어 벡터
  장점 : 소규모 데이터에서도 좋은 성능
Gensim Word2Vec, FastText 벡터 로딩
    Word2Vec : 단어간 의미적 관계학습 ,CBOW(주변단어로  중심단어 예측) , skip-gram(중심단어로 주변단어를 예측)
    장점 : 의미적 유사성 잘 포착(왕 - 여왕)
    단점 : OOV는 처리 불가
    FastText : Word2Vec을 확장한 모델 서브워드(n-gram)로 나눠서 학습
    ex : 영화관 - 영 영화 화관 관 서브워드로 분해, 각 서브워드의 벡터로 합산
    장점 : oov단어도 서브워드로 벡터처리가 가능
    특히 한국어처럼 형태소가 복잡한 언어에 유리
Embedding layer에  가중치 삽입
  word2vec or FastText 벡터를 nn.Embdding 레이어의 초기 가중치로  설정
  프로세스
    데이터셋의 단어사전 생성
    word2vec/fasttest에서 단어 벡터를 가져옴
    임베딩 행렬생성
    nn.embedding에 삽입, 미세조정(fine-tune) 또는 학습고정(freeze)
```


In [ ]:
!pip uninstall numpy
!pip install numpy==1.24.3

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/lib/python3.11/dist-packages/numpy-1.26.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
albumentati

In [ ]:
!pip install konlpy
!pip install gensim

In [ ]:
# 라이브러리
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, FastText

In [ ]:
# 데이터 네이버영화 감성분석 train 데이터만 가져오고 전처리 분할
!wget http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt

--2025-04-25 07:45:56--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 3.5.184.239, 3.5.186.53, 3.5.188.225, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|3.5.184.239|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  9.07MB/s    in 1.5s    

2025-04-25 07:45:58 (9.07 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]



In [ ]:
df = pd.read_csv('ratings_train.txt', sep='\t')
df = df[:10000]
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 전처리
# 형태소분석 어간 추출
okt = Okt()
X = df.document.apply(lambda x : okt.morphs(x, stem=True)).to_list()

In [ ]:
len(X)

10000

In [ ]:
# 형태소분류(어간)가 된 단어리스트 중에서
# 1한글만 그리고 2글자 이상만 필터링(추출)
# r'[^ㄱ-히가-힣\s]+'  한글 및 공백이 연속적으로 표현된 문자을 ^ 제외한다
# r'[가-힣]+'
import re
# [word for words in X for word in words if re.fullmatch(r'[가-힣]+', word)]

X_lists = []
for words in X:
  temp_list = []
  for word in words:
    if len(word)>=2 and re.fullmatch(r'[가-힣]+', word):
      temp_list.append(word)
  X_lists.append(temp_list)

X = X_lists.copy()
y = df.label.to_list()
len(X), len(y)

(10000, 10000)

In [ ]:
# 정수인덱싱 단어사전
# X, y데이터를 분할
# train데이터를기준으로 key:value 형태의 단어사전을 만든다. 이때 맨앞에는 <'PAD'> : 0, '<unk>':1,
# train데이터 있는 단어들을 정수인덱싱
# 학습을위해서 데이터를 Longtensor로 변경

AttributeError: 'list' object has no attribute 'shape'